<a href="https://colab.research.google.com/github/sonaljain2212/Healthcare-Entity-and-Relationship-Extraction-/blob/main/NER_preprocsessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! pip install spacy==2.3.2
! pip install scispacy==0.2.5
! pip install transformers==3.0.2
! pip install numpy==1.19.2
! pip install seqeval==1.2.2
! pip install fastapi==0.61.2
! pip install uvicorn==0.12.2
! pip install pandas==1.1.2
! pip install filelock==3.0.12
! pip install pydantic==1.7.2
! pip install allennlp==1.1.0
! pip install scikit-learn==0.23.2
! pip install networkx==2.5
! pip install matplotlib==3.3.1
! pip install seaborn==0.11.0
! pip install gunicorn==20.0.4
! pip install uvicorn==0.12.2

     |████████████████████████████████| 9.9MB 10.1MB/s 
     |████████████████████████████████| 2.1MB 38.9MB/s 
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 13.5MB 9.7MB/s 
     |████████████████████████████████| 71kB 2.3MB/s 
     |████████████████████████████████| 194kB 45.3MB/s 
     |████████████████████████████████| 778kB 14.3MB/s 
     |████████████████████████████████| 870kB 48.0MB/s 
     |████████████████████████████████| 1.2MB 41.6MB/s 
     |████████████████████████████████| 3.0MB 43.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=38d7dab0599e8c20daedc40ecbe385723592d1a08368bfbab14492d85e1d14fb
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3

     |████████████████████████████████| 51kB 4.5MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16172 sha256=1a14ed677dbf2c309e656a389084e12a8fd70b5a36042845292c85f3c24b5f78
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval
     |████████████████████████████████| 51kB 5.0MB/s 
     |████████████████████████████████| 10.1MB 19.9MB/s 
     |████████████████████████████████| 61kB 6.8MB/s 
     |████████████████████████████████| 51kB 4.9MB/s 
     |████████████████████████████████| 61kB 6.2MB/s 
     |████████████████████████████████| 10.5MB 12.9MB/s 
  Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5


     |████████████████████████████████| 9.1MB 17.9MB/s 
  Found existing installation: pydantic 1.8.1
    Uninstalling pydantic-1.8.1:
      Successfully uninstalled pydantic-1.8.1
     |████████████████████████████████| 491kB 17.3MB/s 
     |████████████████████████████████| 133kB 51.3MB/s 
     |████████████████████████████████| 266kB 52.6MB/s 
     |████████████████████████████████| 748.8MB 19kB/s 
     |████████████████████████████████| 122kB 35.7MB/s 
     |████████████████████████████████| 7.4MB 38.9MB/s 
     |████████████████████████████████| 81kB 7.8MB/s 
  Created wheel for overrides: filename=overrides-3.1.0-cp37-none-any.whl size=10174 sha256=7b50827779bb936c76a34faee6de47064d12e0eb3533408d41cc67a66c4584d4
  Stored in directory: /root/.cache/pip/wheels/5c/24/13/6ef8600e6f147c95e595f1289a86a3cc82ed65df57582c65a9
  Created wheel for jsonnet: filename=jsonnet-0.17.0-cp37-cp37m-linux_x86_64.whl size=3388784 sha256=63148e49287aecca473a16343aaa58e53934413ee16d95758d577b12bcb891f5

     |████████████████████████████████| 286kB 17.3MB/s 
  Found existing installation: seaborn 0.11.1
    Uninstalling seaborn-0.11.1:
      Successfully uninstalled seaborn-0.11.1
     |████████████████████████████████| 81kB 7.1MB/s 


In [ ]:
%cd /content/drive/MyDrive/NER

from utils import read_data, save_pickle, read_ade_data
from biobert_ner.utils_ner import generate_input_files
from biobert_re.utils_re import generate_re_input_files
from typing import List, Iterator, Dict
import warnings
import os

labels = ['B-DRUG', 'I-DRUG', 'B-STR', 'I-STR', 'B-DUR', 'I-DUR',
          'B-ROU', 'I-ROU', 'B-FOR', 'I-FOR', 'B-ADE', 'I-ADE',
          'B-DOS', 'I-DOS', 'B-REA', 'I-REA', 'B-FRE', 'I-FRE', 'O']



def ner_generator(files: Dict[str, tuple],max_seq_len) -> None:
    """Generates files for NER"""
    # Generate train, dev, test files
    for filename, data in files.items():
        generate_input_files(ehr_records=data[0], ade_records=data[1],
                             filename="dataset/" + filename + '.' + "txt",
                             max_len=max_seq_len, sep=" ")
        save_pickle("dataset/" + filename, {"EHR": data[0], "ADE": data[1]})

    # Generate labels file
    with open("dataset/" + 'labels.txt', 'w') as file:
        output_labels = map(lambda x: x + '\n', labels)
        file.writelines(output_labels)

    filenames = [name for files in map(
        lambda x: [x + '.' + "txt", x + '.pkl'],
        list(files.keys()))
                 for name in files]

    print("\nGenerating files successful. Files generated: ",
          ', '.join(filenames), ', labels.txt', sep=" ")


def re_generator(files: Dict[str, tuple]):
    """Generates files for RE"""
    for filename, data in files.items():
        generate_re_input_files(ehr_records=data[0], ade_records=data[1],
                                filename="dataset/" + filename + '.' + "txt",
                                max_len=512, sep=" ",
                                is_test=data[2], is_label=data[3])

    save_pickle("dataset/" + 'train', {"EHR": files['train'][0], "ADE": files['train'][1]})
    save_pickle("dataset/" + 'test',  {"EHR": files['test'][0],  "ADE": files['test'][1]})

    print("\nGenerating files successful. Files generated: ",
          'train.tsv,', 'dev.tsv,', 'test.tsv,',
          'test_labels.tsv,', 'train_rel.pkl,', 'test_rel.pkl,', 'test_labels_rel.pkl', sep=' ')

/content/drive/MyDrive/NER


In [ ]:
from transformers import AutoTokenizer
biobert = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")

# args.max_seq_len -= biobert.num_special_tokens_to_add()
# tokenizer = biobert.tokenize

max_seq_len = 512

max_seq_len -= biobert.num_special_tokens_to_add()
tokenizer = biobert.tokenize
ade_train_dev = None
ade_train = None
ade_devel = None

print("\nReading data\n")
train_dev, test = read_data(data_dir="data/",tokenizer=tokenizer, verbose=1)
    



Reading data

Train data:
Progress: [====================] 303/303

Test Data:
Progress: [====================] 202/202

In [ ]:
# Data is already shuffled, just split for dev set
dev_split_idx = int((1 - 0.1) * len(train_dev))
train = train_dev[:dev_split_idx]
devel = train_dev[dev_split_idx:]



In [ ]:
files = {'train': (train, ade_train), 'train_dev': (train_dev, ade_train_dev),'devel': (devel, ade_devel), 'test': (test, None)}

ner_generator(files,max_seq_len)

In [ ]:
%cd /content/drive/MyDrive/NER
!python generate_data.py \
    --task ner \
    --input_dir data/ \
    --target_dir dataset/ \
    --max_seq_len 512 \
    --dev_split 0.1 \
    --tokenizer biobert-base \
    --ext txt \
    --sep " " \

/content/drive/MyDrive/NER
2021-03-21 02:03:47.646422: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Downloading: 100% 313/313 [00:00<00:00, 244kB/s]
Downloading: 100% 213k/213k [00:00<00:00, 4.04MB/s]

Reading data

Train data:
Progress: [====================] 303/303

Test Data:
Progress: [====================] 202/202

Data successfully saved in dataset/train.txt
Variable successfully saved in dataset/train.pkl
Data successfully saved in dataset/train_dev.txt
Variable successfully saved in dataset/train_dev.pkl
Data successfully saved in dataset/devel.txt
Variable successfully saved in dataset/devel.pkl
Data successfully saved in dataset/test.txt
Variable successfully saved in dataset/test.pkl

Generating files successful. Files generated: train.txt, train.pkl, train_dev.txt, train_dev.pkl, devel.txt, devel.pkl, test.txt, test.pkl, labels.txt


In [ ]:
files = {'train': (train, ade_train, False, True), 'dev': (devel, ade_devel, False, True),
                 'test': (test, None, True, False), 'test_labels': (test, None, True, True)}

re_generator(files)


Variable successfully saved in dataset/train_rel.pkl
Variable successfully saved in dataset/dev_rel.pkl
Variable successfully saved in dataset/test_rel.pkl
Variable successfully saved in dataset/test_labels_rel.pkl
Variable successfully saved in dataset/train.pkl
Variable successfully saved in dataset/test.pkl

Generating files successful. Files generated:  train.tsv, dev.tsv, test.tsv, test_labels.tsv, train_rel.pkl, test_rel.pkl, test_labels_rel.pkl
